In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data_folder_path = '/content/drive/My Drive/adAuditing/personas_new/age'

In [3]:
import json, os

reference: https://www.analyticsvidhya.com/blog/2022/03/keyword-extraction-methods-from-documents-in-nlp/

In [4]:
import spacy
from collections import Counter
from string import punctuation
nlp = spacy.load("en_core_web_sm")
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result


In [20]:
def extractKeywords():
    res = {}
    for file_name in os.listdir(data_folder_path):
        if not file_name.endswith('.json'): continue
        with open(os.path.join(data_folder_path, file_name), 'r') as file:
            data = json.load(file)['data']
            persona = file_name.split('.')[0]
            res[persona] = {}
            for bh in data['browsing_history']:
                keywords = get_hotwords(bh['title'])
                for kw in keywords:
                    if kw in res[persona]:
                        res[persona][kw]['browsing_history'].add(bh['id'])
                    else:
                        res[persona][kw] = {'browsing_history': {bh['id']}, 'schedule': set(), 'facebookPostsList': set()}
            for sch in data['schedule']:
                keywords = get_hotwords(sch['event'])
                for kw in keywords:
                    if kw in res[persona]:
                        res[persona][kw]['schedule'].add(sch['id'])
                    else:
                        res[persona][kw] = {'browsing_history': set(), 'schedule': {sch['id']}, 'facebookPostsList': set()}
    print(res)
    return res

# {pid1 : {kw1 : {'browsing_history' : {id1, id2...}, {'schedule' : {id1, id2...}}, {'facebookPostsList' : {id1, id2...}}}, kw2: ...}, pid2 : {...}, pid3 : {...}}


            # for pc in data['facebookPostsList']:
            #   passage_embedding = model.encode(pc['content'])
            #   if util.dot_score(query_embedding, passage_embedding) > THRESHOLD:
            #     result['facebookPostsList'].append(pc['id'])


In [21]:
all_persona_keywords = extractKeywords()

{'4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a': {'psychology': {'browsing_history': {0, 35, 38, 10, 11, 12, 45, 46, 30}, 'schedule': set(), 'facebookPostsList': set()}, 'today': {'browsing_history': {0, 35, 45, 46}, 'schedule': set(), 'facebookPostsList': set()}, 'therapist': {'browsing_history': {0}, 'schedule': set(), 'facebookPostsList': set()}, 'mental': {'browsing_history': {1, 4, 5, 13, 45, 22, 55, 59, 29}, 'schedule': set(), 'facebookPostsList': set()}, 'health': {'browsing_history': {1, 4, 5, 13, 22, 55, 29}, 'schedule': set(), 'facebookPostsList': set()}, 'blogs': {'browsing_history': {1, 13}, 'schedule': set(), 'facebookPostsList': set()}, 'stress': {'browsing_history': {8, 2, 27}, 'schedule': set(), 'facebookPostsList': set()}, 'anxiety': {'browsing_history': {2}, 'schedule': set(), 'facebookPostsList': set()}, 'best': {'browsing_history': {3, 6, 70, 40, 9, 60, 17, 50, 21, 23, 25, 28, 31}, 'schedule': set(), 'facebookPostsList': set()}, 'online'

In [22]:
def get_common_keywords(all_keywords):
    common_keywords = None

    for persona in all_keywords:
        keywords = set(all_keywords[persona].keys())
        if common_keywords is None:
            common_keywords = keywords
        else:
            common_keywords = common_keywords.intersection(keywords)
    print(f"common kws: {len(common_keywords)}")
    return common_keywords

In [36]:
def get_distinct_keywords(all_keywords, common_keywords, person_id_lst):
    res = {}
    for pid in person_id_lst:
        res[pid] = set(all_keywords[pid].keys()) - common_keywords
    return res
    # {pid1: {w1, w2, ...}, pid2: {...}, ...}


In [33]:
pid_20 = '4b81a719-4d63-4a0d-92be-5087c71a40b0_'
pid_45 = '4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a'
pid_80 = '4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e'

In [35]:
common_keywords = get_common_keywords(all_persona_keywords)
distinct_keywords = get_distinct_keywords(all_persona_keywords, common_keywords, [pid_20, pid_45, pid_80])

common kws: 82


reference: https://github.com/alexandres/lexvec?tab=readme-ov-file#pre-trained-vectors

In [10]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/adAuditing/lexvec.enwiki+newscrawl.300d.W.pos.vectors', binary=False)

In [85]:
def get_vec_word_dic(distinct_keywords, model):
    vec_word_dic = {}       # {pid1: {vec1: word1, ...}, ...}
    ignored_words = set()
    for pid, words in distinct_keywords.items():
        vec_word_dic[pid] = {}
        for word in words:
            try:
                vec_word_dic[pid][tuple(model[word])] = word
            except KeyError:
                ignored_words.add(word)
        print(f'encode {len(vec_word_dic[pid])} words for {pid}')
    print(f'{len(ignored_words)} words ignored: {ignored_words}')
    return vec_word_dic, ignored_words

In [86]:
vec_word_dic, ignored_words = get_vec_word_dic(distinct_keywords, model)

encode 23 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_
encode 48 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_a120fce9-2511-422b-9c80-cb739e59ef6a
encode 112 words for 4b81a719-4d63-4a0d-92be-5087c71a40b0_199bea01-2cba-4381-9980-fad5a40da13e
3 words ignored: {'volunteermatch', 'ravelry', 'freemake'}


In [87]:
from sklearn.neighbors import LocalOutlierFactor

In [126]:
def get_outliers(vec_word_dic, pid):

    all_vectors = list(vec_word_dic[pid].keys())
    length = len(all_vectors)     # all_vectors[0:length] are vectors of target pid
    # print(length)
    for id, word_dic in vec_word_dic.items():
        if id != pid:
            all_vectors += list(word_dic.keys())
    # print(len(all_vectors))

    # Fit LocalOutlierFactor model
    lof_model = LocalOutlierFactor(n_neighbors=40, contamination=0.2)
    lof_model.fit(all_vectors)

    # Predict outliers
    outlier_labels = lof_model.fit_predict(all_vectors)

    outlier_vectors_set = [word for idx, word in enumerate(all_vectors[0:length]) if outlier_labels[idx] == -1]
    print(len(outlier_vectors_set), 'outliers found')

    return {vec_word_dic[pid][vec] for vec in outlier_vectors_set}

In [127]:
get_outliers(vec_word_dic, pid_20)

4 outliers found


{'degree', 'hangout', 'reddit', 'virtual'}

In [128]:
get_outliers(vec_word_dic, pid_45)

4 outliers found


{'laundry', 'mattresses', 'therapy', 'wellness'}

In [129]:
get_outliers(vec_word_dic, pid_80)

29 outliers found


{'allrecipes',
 'arrivals',
 'beginners',
 'catalog',
 'cup',
 'discussion',
 'forecast',
 'gatsby',
 'goodreads',
 'grandchildren',
 'housekeeping',
 'knitting',
 'maps',
 'mattresses',
 'meetup',
 'michaels',
 'mockingbird',
 'oregon',
 'oregonian',
 'portland',
 'samantha',
 'scarf',
 'scarves',
 'spruce',
 'therapy',
 'virtual',
 'wellness',
 'yarn',
 'zoom'}